In [2]:
import os, shutil
from keras import layers
from keras import models
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import shuffle


This is kaglle  Jellyfish Image Dataset (https://www.kaggle.com/datasets/anshtanwar/jellyfish-types/data).
This dataset contains 900 images of jellyfish belonging to six different categories and species: mauve stinger jellyfish, moon jellyfish, barrel jellyfish, blue jellyfish, compass jellyfish, and lion’s mane jellyfish. You can apply ML techniques to gain insights into jellyfish classification, species identification, and color analysis.

**Moon jellyfish (Aurelia aurita)**: Common jellyfish with four horseshoe-shaped gonads visible through the top of its translucent bell. It feeds by collecting medusae, plankton, and mollusks with its tentacles.
**Barrel jellyfish (Rhizostoma pulmo)**: Largest jellyfish found in British waters, with a bell that can grow up to 90 cm in diameter. It feeds on plankton and small fish by catching them in its tentacles.
**Blue jellyfish (Cyanea lamarckii)**: Large jellyfish that can grow up to 30 cm in diameter. It feeds on plankton and small fish by catching them in its tentacles.
**Compass jellyfish (Chrysaora hysoscella)**: Named after the brown markings on its bell that resemble a compass rose. It feeds on plankton and small fish by catching them in its tentacles.
**Lion’s mane jellyfish (Cyanea capillata)**: Largest jellyfish in the world, with a bell that can grow up to 2 meters in diameter and tentacles that can reach up to 30 meters in length. It feeds on plankton and small fish by catching them in its tentacles.
**Mauve stinger (Pelagia noctiluca)**: Small jellyfish with long tentacles and warty structures on its bell full of stinging cells. It feeds on other small jellyfish and oceanic sea squirts.

In [3]:
os.getcwd()

'C:\\Users\\Peter\\DataspellProjects\\kaggle\\jellyfish_image_classification'

In [4]:
TRAIN_DIR = 'C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train'
TEST_DIR = 'C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/test'
IMG_SIZE = 224# our images are 224x224x3


In [59]:
#Converting the output into one-hot format
def label_img(img):
    word_label = img.split('_')[0]
    if word_label == 'barrel': return [1,0,0,0,0,0]
    elif word_label == 'blue': return [0,1,0,0,0,0]
    elif word_label == 'compass': return [0,0,1,0,0,0]
    elif word_label == 'lions': return [0,0,0,1,0,0]
    elif word_label == 'mauve': return [0,0,0,0,1,0]
    elif word_label == 'Moon': return [0,0,0,0,0,1]

In [64]:
#A function to read training images from folder
def create_train_data():
    training_data = []
    for fold in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(fold)
        path = TRAIN_DIR + '/' + fold
        print(path)
        for i in fold:
            path_img = path + 
            img = cv2.imread(path,1)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            training_data.append([np.array(img),np.array(label)])
        
    shuffle(training_data)
    images, labels = zip(*training_data)  # Разделяем изображения и метки

    # np.save('train_data_mc.npy', training_data)
    np.savez('train_data_mc.npz', images=np.array(images), labels=np.array(labels))
    return training_data

In [65]:
ctrain_data = create_train_data()

  0%|          | 0/6 [00:00<?, ?it/s]

C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train/barrel_jellyfish


UnboundLocalError: local variable 'img' referenced before assignment

In [60]:
label_img('C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train/barrel_jellyfish')
print(label_img)

<function label_img at 0x00000204685EDF30>


In [58]:
import os
import cv2

def read_images_from_folder(folder_path):
    images = []
    labels = []
    training_data = []
    for class_name in os.listdir(folder_path):
        # label = label_img(class_name)
        class_path = os.path.join(folder_path, class_name)

        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                    if img is not None:
                        images.append(img)
                        label = label_img(img)
                        # labels.append(class_name)
                        training_data.append([np.array(img),np.array(labels)])
                    else:
                        print(f"Error reading image: {img_path}, img is None")
                except Exception as e:
                    print(f"Error loading image: {img_path}, {e}")

    return images, labels

# Пример использования
train_folder_path = 'C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train'
images, labels = read_images_from_folder(train_folder_path)

# images содержит считанные изображения, labels содержит соответствующие им названия классов


Error loading image: C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train\barrel_jellyfish\01.jpeg, 'numpy.ndarray' object has no attribute 'split'
Error loading image: C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train\barrel_jellyfish\02.jpg, 'numpy.ndarray' object has no attribute 'split'
Error loading image: C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train\barrel_jellyfish\03.jpg, 'numpy.ndarray' object has no attribute 'split'
Error loading image: C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train\barrel_jellyfish\04.jpg, 'numpy.ndarray' object has no attribute 'split'
Error loading image: C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train\barrel_jellyfish\05.jpg, 'numpy.ndarray' object has no attribute 'split'
Error loading image: C:/Users/Pete

In [51]:
images

[array([[[ 89,  62,   0],
         [ 89,  64,   0],
         [ 91,  66,   4],
         ...,
         [116,  94,  23],
         [115,  92,  24],
         [114,  91,  23]],
 
        [[ 90,  63,   0],
         [ 89,  64,   0],
         [ 91,  66,   4],
         ...,
         [116,  93,  25],
         [115,  92,  24],
         [114,  91,  23]],
 
        [[ 90,  63,   0],
         [ 90,  65,   1],
         [ 91,  66,   4],
         ...,
         [116,  93,  25],
         [115,  92,  24],
         [114,  91,  23]],
 
        ...,
 
        [[  0,  15,  22],
         [  0,  12,  20],
         [  0,  10,  16],
         ...,
         [  8,  28,  39],
         [  7,  23,  35],
         [  5,  21,  33]],
 
        [[  0,  15,  22],
         [  0,  17,  22],
         [  1,  15,  21],
         ...,
         [ 11,  36,  46],
         [ 10,  31,  39],
         [ 10,  29,  37]],
 
        [[  0,  12,  18],
         [  0,  18,  23],
         [  4,  20,  26],
         ...,
         [  9,  37,  44],
  

In [32]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)  # Используйте os.path.join для корректного формирования пути
        print(path)  # Добавьте эту строку для проверки пути

        try:
            img = cv2.imread(path, 1)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                plt.imshow(img)
                training_data.append([np.array(img), np.array(label)])
            else:
                print(f"Error reading image: {path}, img is None")
        except Exception as e:
            print(f"Error loading image: {path}, {e}")

    shuffle(training_data)
    images, labels = zip(*training_data)
    np.savez('train_data_mc.npz', images=np.array(images), labels=np.array(labels))
    return training_data


In [35]:
cv2.imread('C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train/barrel_jellyfish/01.jpeg')

array([[[ 89,  62,   0],
        [ 89,  64,   0],
        [ 91,  66,   4],
        ...,
        [116,  94,  23],
        [115,  92,  24],
        [114,  91,  23]],

       [[ 90,  63,   0],
        [ 89,  64,   0],
        [ 91,  66,   4],
        ...,
        [116,  93,  25],
        [115,  92,  24],
        [114,  91,  23]],

       [[ 90,  63,   0],
        [ 90,  65,   1],
        [ 91,  66,   4],
        ...,
        [116,  93,  25],
        [115,  92,  24],
        [114,  91,  23]],

       ...,

       [[  0,  15,  22],
        [  0,  12,  20],
        [  0,  10,  16],
        ...,
        [  8,  28,  39],
        [  7,  23,  35],
        [  5,  21,  33]],

       [[  0,  15,  22],
        [  0,  17,  22],
        [  1,  15,  21],
        ...,
        [ 11,  36,  46],
        [ 10,  31,  39],
        [ 10,  29,  37]],

       [[  0,  12,  18],
        [  0,  18,  23],
        [  4,  20,  26],
        ...,
        [  9,  37,  44],
        [  8,  31,  39],
        [  7,  28,  36]]

In [39]:
train_data = create_train_data()

  0%|          | 0/6 [00:00<?, ?it/s]

C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/Train/barrel_jellyfish


error: OpenCV(4.6.0) C:\b\abs_f8n1j3l9l0\croot\opencv-suite_1691622637237\work\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [13]:
#A function to read training images from folder
def create_test_data():
    test_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        label = label_img(img)
        path = TEST_DIR + '/' + img
        print(path)
        img = cv2.imread(path,1)
        # img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        test_data.append([np.array(img),np.array(label)])
    
    shuffle(test_data)
    images, labels = zip(*test_data)  # Разделяем изображения и метки

    np.savez('test_data_mc.npz', images=np.array(images), labels=np.array(labels))
    return test_data

In [12]:
test_data = create_test_data()


100%|██████████| 6/6 [00:00<00:00, 2998.07it/s]

C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/test/barrel_jellyfish
C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/test/blue_jellyfish
C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/test/compass_jellyfish
C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/test/lions_mane_jellyfish
C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/test/mauve_stinger_jellyfish
C:/Users/Peter/DataspellProjects/kaggle/jellyfish_image_classification/data/Train_Test_Valid/test/Moon_jellyfish


NameError: name 'images' is not defined

In [10]:
plt.imshow(np.array(train_data[150][0]))


IndexError: list index out of range

In [19]:
train_data

[[array(None, dtype=object), array([0, 0, 0, 0, 0, 1])],
 [array(None, dtype=object), array([0, 0, 0, 1, 0, 0])],
 [array(None, dtype=object), array([0, 1, 0, 0, 0, 0])],
 [array(None, dtype=object), array([0, 0, 1, 0, 0, 0])],
 [array(None, dtype=object), array([1, 0, 0, 0, 0, 0])],
 [array(None, dtype=object), array([0, 0, 0, 0, 1, 0])]]